In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json



In [2]:
# Get Statistic Area level 2 information from csv file
sa2_df = pd.read_csv("resources/SA2_2016_AUST.csv")

# Filter the information to show Greater Perth Only
sa2_perth_df = sa2_df[sa2_df["GCCSA_NAME_2016"] == "Greater Perth"]

sa2_perth_df.head()


,SA2_MAINCODE_2016,SA2_5DIGITCODE_2016,SA2_NAME_2016,SA3_CODE_2016,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,GCCSA_CODE_2016,GCCSA_NAME_2016,STATE_CODE_2016,STATE_NAME_2016,AREA_ALBERS_SQKM
1768,502011021,51021,Dawesville - Bouvard,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,97.9248
1769,502011022,51022,Falcon - Wannanup,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,9.5893
1770,502011023,51023,Greenfields,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,9.9609
1771,502011024,51024,Halls Head - Erskine,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,14.2469
1772,502011025,51025,Mandurah,50201,Mandurah,502,Mandurah,5GPER,Greater Perth,5,Western Australia,8.3954


In [3]:
# Store information into the dictionaly
repay_dict = {"SA2 Name":[],"$1-$149":[], "$150-$299":[], "$300-$449":[], "$450-$599":[],
              "$600-$799":[], "$800-$999":[], "$1,000-$1,199":[], "$1,200-$1,399":[],
              "$1,400-$1,599":[], "$1,600-$1,799":[], "$1,800-$1,999":[], "$2,000-$2,199":[],
              "$2,200-$2,399":[], "$2,400-$2,599":[], "$2,600-$2,999":[], "$3,000-$3,999":[], "$4,000-$4,999":[], "$5,000 and over":[]}

#   Data:  One family household with only family members present
for code in sa2_perth_df["SA2_MAINCODE_2016"]:
    sa2_code = str(code)
    report_range = np.arange(2,20)
    repay_dict["SA2 Name"].append(sa2_perth_df.loc[sa2_perth_df["SA2_MAINCODE_2016"]==code,"SA2_NAME_2016"].values[0])

# Report type start with 03 and finish 22
    for number in report_range:
        report_type = "{0:0=2d}".format(number)
        abs_url = f"http://stat.data.abs.gov.au/sdmx-json/data/ABS_C16_T25_TS_SA/{report_type}.TOT.5.SA2.{code}/all?detail=Full&dimensionAtObservation=AllDimensions&startPeriod=2016"
        response = requests.get(abs_url)
        data = response.json()
#abs data name is set as key of dictionary, and data was included in observations
        repay_dict[f'{data["structure"]["dimensions"]["observation"][0]["values"][0]["name"]}'].append(data["dataSets"][0]["observations"]["0:0:0:0:0:0"][0])




In [43]:
repay_df = pd.DataFrame(repay_dict)

In [44]:
# Rename the columns to see the data more easily
repay_df.columns = ["SA2 Name", "1-149", "150-299", "300-449", "450-599", "600-799",
                    "800-999", "1,000-1,199", "1,200-1,399", "1,400-1,599", "1,600-1,799",
                    "1,800-1,999", "2,000-2,199", "2,200-2,399", "2,400-2,599", "2,600-2,999",
                    "3,000-3,999", "4,000-4,999", "5,000 and over"]

# To calculate the median from histogram 
bin_size = [150, 150, 150, 150, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 400, 1000, 1000, 1000]
bin_left = [1, 150, 300, 450, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600, 3000, 4000, 5000]

# Median from Histogram

Use interpolated Median


reference url
https://www.vivaxsolutions.com/maths/allnrintpltn.aspx

https://math.stackexchange.com/questions/2591946/how-to-find-median-from-a-histogram

Median=   l+  (n/2−F)/fm *w
where l is the lower border of the median group, F is the cumulative frequency up to the median group, fm is the frequency of the median group, w is the width of the median group. 

In [45]:
repay_df["Total"] = repay_df.iloc[:,1:19].sum(axis=1)

In [46]:
clean_df = repay_df[repay_df["Total"] > 100]

In [47]:
clean_df["Median"] =""
clean_df = clean_df.reset_index()
clean_df = clean_df.iloc[:,1:]

clean_df.columns

C:\Users\khiro\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Index(['SA2 Name', '1-149', '150-299', '300-449', '450-599', '600-799',
       '800-999', '1,000-1,199', '1,200-1,399', '1,400-1,599', '1,600-1,799',
       '1,800-1,999', '2,000-2,199', '2,200-2,399', '2,400-2,599',
       '2,600-2,999', '3,000-3,999', '4,000-4,999', '5,000 and over', 'Total',
       'Median'],
      dtype='object')

In [48]:
#Calculate Median from histgram value (search the index which exceeds (total number /2) )
for index_k,row in clean_df.iterrows():
    counter = 0
    i = 1
    while counter < (row[19]+1)/2:
        counter = counter + row[i]
        i = i + 1
# Linear interpolation   lower border + ((total number +1)/2 - frequency up to the median group) / frequency of median group  * width of median group
    median = bin_left[i-2] + ((row[19]+1)/2 - (counter - row[i-1])) * bin_size[i-2] / (row[i-1])
    clean_df.iloc[index_k,20] = median
    

In [51]:
clean_df.to_csv("resources/mortgage_repayment.csv")

In [53]:
# get suburb name (SA2) and Median
clean_df[["SA2 Name","Median"]]

,SA2 Name,Median
0,Dawesville - Bouvard,2032.89
1,Falcon - Wannanup,1925.33
2,Greenfields,1562.35
3,Halls Head - Erskine,1976.19
4,Mandurah,1546.24
...,...,...
146,Rockingham,1715.13
147,Safety Bay - Shoalwater,1850.43
148,Singleton - Golden Bay - Secret Harbour,2136.53
149,Waikiki,1795.65


In [65]:
repay_median = clean_df[["SA2 Name","Median"]]

In [66]:
# Calclate Weekly data from monthly data
repay_median["weekly repayment(median)"] = repay_median["Median"] * 12 / 365 * 7


C:\Users\khiro\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [68]:
repay_median.to_csv("resources/mortgage_median.csv")